#FSN Analysis

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [2]:
#Load the data
#import drive
from google.colab import drive
drive.mount('/content/drive') 

Mounted at /content/drive


In [3]:
data = pd.read_csv('/content/drive/My Drive/Inventory Management/Input/FSN_data.csv')

In [4]:
data.head()

,No_item,Brand,Dept,Stok_Awal,Stok_Akhir,Demand
0,100002,LIST,PDG,0.0,0.0,0.0
1,100003,ATOOL,PDG,40.0,38.0,2.0
2,100008,ATOOL,PDG,16.0,13.0,3.0
3,100011,ATOOL,PDG,10.0,10.0,0.0
4,100012,ATOOL,PDG,39.0,39.0,0.0


In [5]:
def FSN_segmentation(tor):
    '''
    Creates the 3 classes F, S, and N based 
    on TOR / Turn Over Ratio (F > 3, 1 < S < 3, N < 1)
    '''
    if tor > 3 and not 'inf' and not '-inf':
        return 'F'
    elif tor > 1 and tor < 3:
        return 'S'
    elif tor < 1:
        return 'N'
    elif tor == 'inf':
        return '0'
    elif tor == '-inf':
        return '0'

def Demand_Class(dem):
    if dem == 0:
      return '0'

In [6]:
# take a subset of the data, we need to use the price and the quantity of each item
data_sub = data[['No_item','Brand','Dept','Stok_Awal', 'Stok_Akhir', 'Demand']]

# create the column of the Average Inventory
data_sub['Avg_inv'] = (data_sub['Stok_Awal'] + data_sub['Stok_Akhir']) / 2

# Average inventory total = total avg inv / 12 bulan
data_sub['Avg_inv_tot'] = data_sub['Avg_inv'] / 12

data_sub['Demand'] == data_sub['Demand']

#data_sub['TOR'] = data_sub['Demand'].apply(Demand_Class)
data_sub['Avg_inv_tot'] / data_sub['Demand']
data_sub['TOR'] = data_sub['Avg_inv_tot'] / data_sub['Demand']

In [7]:
data_sub['Class_FSN'] = data_sub['TOR'].apply(FSN_segmentation)


In [9]:
data_sub.head()

,No_item,Brand,Dept,Stok_Awal,Stok_Akhir,Demand,Avg_inv,Avg_inv_tot,TOR,Class_FSN
0,100002,LIST,PDG,0.0,0.0,0.0,0.0,0.000000,NaN,None
1,100003,ATOOL,PDG,40.0,38.0,2.0,39.0,3.250000,1.625000,S
2,100008,ATOOL,PDG,16.0,13.0,3.0,14.5,1.208333,0.402778,N
3,100011,ATOOL,PDG,10.0,10.0,0.0,10.0,0.833333,inf,None
4,100012,ATOOL,PDG,39.0,39.0,0.0,39.0,3.250000,inf,None


In [10]:
#Data output: Product recommendationnya gabung dalam satu kolom
df_output = data_sub[['Dept', 'Class_FSN', 'Brand', 'No_item', 'TOR']].drop_duplicates()

In [11]:
#OUTPUT CSV
df_output.to_csv('result_FSN_sti.csv', index=False)

In [12]:

def create_output(print_json=True):
  df_output = data_sub[['Dept', 'Class', 'Brand', 'No_item', 'TOR']].drop_duplicates() \
        .sort_values('Dept').set_index('Dept')
  if print_json:
    df_output.to_json('/content/drive/My Drive/Inventory Management/FSN_Sti_output.json')
    print("An output file can be found in 'output' folder with name 'option1_recommendation.json'")
  return df_output

In [14]:
print(df_output.shape)
df_output.head()

(11552, 5)


,Dept,Class_FSN,Brand,No_item,TOR
0,PDG,None,LIST,100002,NaN
1,PDG,S,ATOOL,100003,1.625000
2,PDG,N,ATOOL,100008,0.402778
3,PDG,None,ATOOL,100011,inf
4,PDG,None,ATOOL,100012,inf


In [15]:
#Format Output
finalList = []
finalDict = {}
grouped = df_output.groupby(['Dept'])
# Group per deptCode
for key, value in grouped:        
      #print(key)
      #print(value)
      deptCode = grouped.get_group(key).reset_index(drop=True)

      deptCodeList= []
      deptCodeDict = {}
      ObjectValue = {}
      #fsn class
      grouped_FSN = deptCode.groupby(['Dept','Class_FSN'])
      FSNObjectValue = {}
      for keyFSN, valueFSN in grouped_FSN:
          FSNList = []
          namaFSN = keyFSN[1]
          FSNCode = grouped_FSN.get_group(keyFSN).reset_index(drop=True)
      
      grouped_Brand = FSNCode.groupby(['Class_FSN','Brand'])
      
      BrandObjectValue = {}
      for keyBrand, valueBrand in grouped_Brand: 
          BrandList= []
          namaBrand = keyBrand[1]
          
          BrandCode = grouped_Brand.get_group(keyBrand).reset_index(drop=True)

          for i_Brand in BrandCode.index:
            BrandDict = {}
            BrandDict['No_item'] = BrandCode.at[i_Brand, 'No_item']
            BrandDict['TOR'] = BrandCode.at[i_Brand, 'TOR']
            BrandList.append(BrandDict)
          BrandObjectValue[namaBrand] = BrandList
      FSNObjectValue[namaFSN] = BrandObjectValue
      # print(BrandObjectValue)
      deptCodeList.append(FSNObjectValue)
      deptCodeDict[key] = deptCodeList
      # print(deptCodeDict)

      finalList.append(deptCodeDict)
import json
print(finalList)

[{'PDG': [{'S': {'AFA ': [{'No_item': 107989, 'TOR': 2.576388888888889}], 'AM ': [{'No_item': 112225, 'TOR': 1.4583333333333333}], 'AND ': [{'No_item': 108070, 'TOR': 2.263888888888889}, {'No_item': 119830, 'TOR': 1.9583333333333333}], 'ATOOL ': [{'No_item': 100003, 'TOR': 1.625}, {'No_item': 100160, 'TOR': 2.2083333333333335}, {'No_item': 100169, 'TOR': 1.125}, {'No_item': 109000, 'TOR': 1.375}], 'BAUT ': [{'No_item': 100482, 'TOR': 1.4583333333333333}], 'BENZ ': [{'No_item': 103098, 'TOR': 1.125}, {'No_item': 107932, 'TOR': 2.4583333333333335}], 'BOSCH ': [{'No_item': 106675, 'TOR': 1.0416666666666667}], 'BREMEN ': [{'No_item': 111552, 'TOR': 1.2083333333333333}], 'DELIZIA ': [{'No_item': 107920, 'TOR': 2.225}], 'ECO ': [{'No_item': 112302, 'TOR': 1.9027777777777777}], 'EURO ': [{'No_item': 105744, 'TOR': 1.8761111111111113}], 'EWIGE ': [{'No_item': 116947, 'TOR': 1.4166666666666667}], 'FERZA ': [{'No_item': 109342, 'TOR': 1.0694444444444444}], 'FIT ': [{'No_item': 102988, 'TOR': 1.2

In [16]:
df_output.dtypes

Dept          object
Class_FSN     object
Brand         object
No_item        int64
TOR          float64
dtype: object

In [17]:

df_output['No_item'] =df_output['No_item'].apply(lambda x:str(x))

In [19]:
#NEWEDIT Format Output Cara 1 : AMBERCODE
finalList = []
finalDict = {}
grouped = df_output.groupby('Dept')
#print(grouped['Dept'].count())
# BLOK A
for key, value in grouped:        
    deptCode = grouped.get_group(key).reset_index(drop=True)
    deptCodeList= []
    deptCodeDict = {}
    ObjectValue = {}
    
    #BLOK B
    grouped_FSN = deptCode.groupby(['Dept','Class_FSN'])
    print(grouped_FSN['Class_FSN'].count())
    FSNObjectValue = {}
    for keyFSN, valueFSN in grouped_FSN:
        FSNList = []
        namaFSN = keyFSN[1]
        FSNCode = grouped_FSN.get_group(keyFSN).reset_index(drop=True)
        
        #BLOK C
        grouped_Brand = FSNCode.groupby(['Class_FSN','Brand'])
        # print(grouped_Brand['Brand'].count())
        # print(grouped_FSN.count())
        BrandObjectValue = {}
        for keyBrand, valueBrand in grouped_Brand: 
            BrandObjectValue = {}
            BrandList= []
            namaBrand = keyBrand[1]
            BrandCode = grouped_Brand.get_group(keyBrand).reset_index(drop=True)
            
            #BLOK D
            for i_Brand in BrandCode.index:
              BrandDict = {}
              BrandDict['No_item'] = BrandCode.at[i_Brand, 'No_item']
              BrandDict['TOR'] = BrandCode.at[i_Brand, 'TOR']
              BrandList.append(BrandDict)          
            BrandObjectValue[namaBrand] = BrandList

        FSNObjectValue[namaFSN] = BrandObjectValue

deptCodeList.append(FSNObjectValue)
deptCodeDict[key] = deptCodeList

finalList.append(deptCodeDict)
import json
print(finalList)

Dept  Class_FSN
PDG   N            2067
      S              83
Name: Class_FSN, dtype: int64
Dept  Class_FSN
PKU   N            493
      S             36
Name: Class_FSN, dtype: int64
[{'PKU': [{'N': {'WALL ': [{'No_item': '104632', 'TOR': 0.016856392294220666}, {'No_item': '104634', 'TOR': 0.012681159420289856}, {'No_item': '104635', 'TOR': 0.019136807817589578}, {'No_item': '104640', 'TOR': 0.10138888888888889}, {'No_item': '107460', 'TOR': 0.04166666666666667}, {'No_item': '110823', 'TOR': 0.041666666666666664}, {'No_item': '119995', 'TOR': 0.0}]}, 'S': {'STI ': [{'No_item': '109741', 'TOR': 1.0416666666666667}, {'No_item': '109787', 'TOR': 2.0}]}}]}]


In [20]:
finalList = []
finalDict = {}
grouped = df_output.groupby(['Dept'])
# Group per deptCode
for key, value in grouped:        
      deptCode = grouped.get_group(key).reset_index(drop=True)
      print(deptCode)

     Dept Class_FSN   Brand No_item       TOR
0     PDG      None   LIST   100002       NaN
1     PDG         S  ATOOL   100003  1.625000
2     PDG         N  ATOOL   100008  0.402778
3     PDG      None  ATOOL   100011       inf
4     PDG      None  ATOOL   100012       inf
...   ...       ...     ...     ...       ...
5771  PDG      None   TANG   120006       NaN
5772  PDG      None  OBENG   120007       NaN
5773  PDG      None    LEM   120009       NaN
5774  PDG      None   KUAS   120010       NaN
5775  PDG      None  KUNCI   120011       NaN

[5776 rows x 5 columns]
     Dept Class_FSN   Brand No_item  TOR
0     PKU      None   LIST   100002  NaN
1     PKU      None  ATOOL   100003  NaN
2     PKU      None  ATOOL   100008  NaN
3     PKU      None  ATOOL   100011  NaN
4     PKU      None  ATOOL   100012  NaN
...   ...       ...     ...     ...  ...
5771  PKU      None   TANG   120006  NaN
5772  PKU      None  OBENG   120007  NaN
5773  PKU      None    LEM   120009  NaN
5774  PKU    

In [22]:
#Save Result
file_name = "resultFSN.json"
with open(file_name, "w") as fid:
  json.dump(finalList, fid)

In [23]:
df_output.dtypes

Dept          object
Class_FSN     object
Brand         object
No_item       object
TOR          float64
dtype: object

In [24]:
df_output['No_item'] =df_output['No_item'].apply(lambda x:str(x))